In [1]:
from llama_cpp import Llama

# Download the model file first to your local directory
model_path = "./Models/finance-chat.Q3_K_M.gguf"  # Example GGUF file

# Initialize the Llama model with the GGUF file
llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # The max sequence length to use
    n_threads=8,  # The number of CPU threads to use
    n_gpu_layers=35  # The number of layers to offload to GPU
)

# Example usage of the model
output = llm(
    "[INST] <<SYS>>\n{system_message}\n<</SYS>>\n{prompt} [/INST]",  # The prompt
    max_tokens=512,  # Maximum number of tokens to generate
    stop=["</s>"],  # Stop token
    echo=True  # Whether to echo the prompt in the output
)

# Print the generated output
print(output)


ModuleNotFoundError: No module named 'llama_cpp'